In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import random_split

In [2]:
print(torch.cuda.is_available())

frac_train = 0.8
batch_size = 64
learning_rate = 0.0001
learning_rates = [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]
optimizers = {
    'SGD': torch.optim.SGD,
    'RMSprop': torch.optim.RMSprop,
    'Adam': torch.optim.Adam,
    #'LBFGS' : torch.optim.LBFGS,
}
epoch_num = 20

True


In [3]:
# Download training data from open datasets.
dataset = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [4]:
dataset.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No


In [5]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['RainToday', 'RainTomorrow']
label_encoder = LabelEncoder()

for column in columns_to_encode:
    dataset[column] = label_encoder.fit_transform(dataset[column])

In [6]:
object_columns = dataset.select_dtypes(include=['object']).columns
dataset = dataset.drop(object_columns, axis=1)

In [7]:
X = dataset.drop(columns=['RainTomorrow'])  # Features
y = dataset['RainTomorrow']  # Target

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y.view(-1, 1)  # Reshape y to have shape [num_samples, 1]
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

custom_dataset = CustomDataset(X_tensor, y_tensor)

In [8]:
train_size = int(0.8 * len(custom_dataset))  # 80% of the data for training
test_size = len(custom_dataset) - train_size  # Remaining 20% for testing

# Split the dataset
train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

# Create DataLoader for training data
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for test data
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 17])
Shape of y: torch.Size([64, 1]) torch.float32


In [10]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Sample layers of Neural Network Class
# nn.Linear(28*28, 512),
# nn.ReLU(),
# nn.Linear(512, 512),
# nn.ReLU(),
# nn.Linear(512, 10)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(17, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.Dropout(0.25),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=17, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): Dropout(p=0.25, inplace=False)
    (6): ReLU()
    (7): Linear(in_features=16, out_features=8, bias=True)
    (8): Dropout(p=0.5, inplace=False)
    (9): ReLU()
    (10): Linear(in_features=8, out_features=1, bias=True)
    (11): Sigmoid()
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
# epochs = 5
for t in range(epoch_num):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:     nan  [   64/116368]
loss:     nan  [ 6464/116368]
loss:     nan  [12864/116368]
loss:     nan  [19264/116368]
loss:     nan  [25664/116368]
loss:     nan  [32064/116368]
loss:     nan  [38464/116368]
loss:     nan  [44864/116368]
loss:     nan  [51264/116368]
loss:     nan  [57664/116368]
loss:     nan  [64064/116368]
loss:     nan  [70464/116368]
loss:     nan  [76864/116368]
loss:     nan  [83264/116368]
loss:     nan  [89664/116368]
loss:     nan  [96064/116368]
loss:     nan  [102464/116368]
loss:     nan  [108864/116368]
loss:     nan  [115264/116368]
Test Error: 
 Accuracy: 4847.3%, Avg loss:      nan 

Epoch 2
-------------------------------
loss:     nan  [   64/116368]
loss:     nan  [ 6464/116368]
loss:     nan  [12864/116368]
loss:     nan  [19264/116368]
loss:     nan  [25664/116368]
loss:     nan  [32064/116368]
loss:     nan  [38464/116368]
loss:     nan  [44864/116368]
loss:     nan  [51264/116368]
loss:     nan  [57664/1

In [15]:
model.eval()    # 평가시에는 dropout이 OFF 된다.
correct = 0
for data, target in test_dataloader:
    data = data.to(device)
    target = target.to(device)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()
print('Test set Accuracy : {:.2f}%'.format(100. * correct / len(test_dataloader.dataset)))

Test set Accuracy : 4847.28%
